In [1]:
%run helper/setup_notebook.ipynb

Successfully connected to sql_lab database.


In [2]:
%%sql 

SELECT *
FROM employee;

emp_id,first_name,last_name,dept_id,manager_id,office_id
1,Sally,Jones,3,2,5
2,Mark,Smith,2,4,3
3,John,Andrews,1,4,3
4,Michelle,Johnson,2,None,5
5,Brian,Grand,2,2,3


In [3]:
%%sql 

SELECT * 
FROM department;

dept_id,dept_name
1,Sales
2,IT
3,Support


##### Display the employee name and count per department.

In [4]:
%%sql 

-- step 1. find the department id and the count of each department
SELECT dept_id, COUNT(*) as dept_count
FROM employee
GROUP BY dept_id;

dept_id,dept_count
3,1
2,3
1,1


In [5]:
%%sql 

-- step 2. run an inner join based on employee's department id 

SELECT e.first_name, 
e.last_name, 
d.dept_count
FROM employee e
INNER JOIN (
    SELECT dept_id, COUNT(*) AS dept_count
    FROM employee 
    GROUP BY dept_id 
) d
ON e.dept_id = d.dept_id

first_name,last_name,dept_count
Sally,Jones,1
Mark,Smith,3
John,Andrews,1
Michelle,Johnson,3
Brian,Grand,3


#### ***`CTE`*** allow us to move the subquery and define it separately. 

In [6]:
%%sql 

WITH dept_count_table AS (
    SELECT dept_id, COUNT(*) AS dept_count
    FROM employee
    GROUP BY dept_id
)
SELECT e.first_name,
e.last_name,
dct.dept_count
FROM employee e
INNER JOIN dept_count_table dct ON e.dept_id = dct.dept_id


first_name,last_name,dept_count
Sally,Jones,1
Mark,Smith,3
John,Andrews,1
Michelle,Johnson,3
Brian,Grand,3


#### We can create multiple CTEs seperated by `comma(,)`

In [7]:
%%sql 

WITH dept_count_table AS(
    SELECT dept_id, COUNT(*) AS dept_count
    FROM employee
    GROUP BY dept_id
),
mng_table AS(
    SELECT CONCAT(mng.first_name, ' ', mng.last_name) AS manager_full_name, 
        mng.emp_id,
        emp.dept_id
    FROM employee mng 
    INNER JOIN employee emp ON mng.emp_id = emp.manager_id
)
SELECT CONCAT(e.first_name,' ',e.last_name) AS employee_full_name,
dct.dept_count,
mt.manager_full_name
FROM employee e
INNER JOIN dept_count_table dct ON e.dept_id = dct.dept_id
LEFT JOIN mng_table mt ON e.dept_id = mt.dept_id AND e.manager_id = mt.emp_id


employee_full_name,dept_count,manager_full_name
Sally Jones,1,Mark Smith
Mark Smith,3,Michelle Johnson
John Andrews,1,Michelle Johnson
Michelle Johnson,3,None
Brian Grand,3,Mark Smith
